<h1>ChatBot</h1>

In this notebook, we will try to implement a chatbot that responds to human language. This chatbot has a set of predefined answers and chooses the most proper one depending on the message it receives.

In [1]:
import json
import numpy as np
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import stopwords
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import random
from torch.optim.lr_scheduler import ExponentialLR

Reading the intents file and creating $(X, y)$ where $X$ represents the sentences and $y$ represents the target classes.

In [2]:
with open('./intents.json', 'r') as f:
    intents = json.load(f)
    

classes = []
patterns = []
target = []

for i in range(len(intents['intents'])):
    classes.append(intents['intents'][i]['tag'])
    for j in range(len(intents['intents'][i]['patterns'])):
        patterns.append(intents['intents'][i]['patterns'][j])
        target.append(intents['intents'][i]['tag'])

classes = np.array(classes)
target = np.array(target)
patterns = np.array(patterns)

In [3]:
# a dictionary that maps encoded classes to their name

target2int = {}

for index, category in enumerate(classes):
    target2int[category] = index
        
y = np.vectorize(target2int.get)(target)

Preprocessing the sentences. This includes:
<ul>
    <li>Removing punctuation</li>
    <li>Lowering each character so we will deal with fewer words</li>
    <li>Tokenizing</li>
    <li>Stemming</li>
</ul>

In [4]:
def preprocess(patterns):
    
    processed_patterns = []
    
    for pattern in patterns:
        
        punct_pattern = ("[,.\"!@#$%^&*(){}?/;`~:<>+=-]")
        punct = re.compile(punct_pattern)
        pattern = re.sub(punct, "", pattern)
        
        pattern = pattern.lower()
        
        pattern = word_tokenize(pattern)
        
        ps = PorterStemmer()

        pattern = [ps.stem(pattern) for pattern in pattern]
        
        pattern = ' '.join(pattern)
                
        processed_patterns.append(pattern)

    return(processed_patterns)

processed_patterns = preprocess(patterns)

Getting a matrix with $TF-IDF$ entries. This matrix will be used as training features.

In [5]:
vectorizer = TfidfVectorizer(min_df=0)   
X = vectorizer.fit_transform(processed_patterns).toarray()

feature_size = X.shape[1]

train_data = np.array(list(zip(X, y)), dtype=object)

Getting a data loader for training data

In [6]:
def vectorize(batch):
    
    X, y = list(zip(*batch))
    
    X, y = np.array(X), np.array(y)
        
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)



batch_size = 20

train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=vectorize)

Simple feed forward neural network to classify each user input.

In [7]:
class FeedForwardClassifier(nn.Module):

    def __init__(self):
        super(FeedForwardClassifier, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(feature_size, 128),
            nn.ReLU(),

            nn.Linear(128,16),
            nn.ReLU(),

            nn.Linear(16, len(classes)),
        )

    def forward(self, X):
        return self.layers(X)   

Hyper parameters

In [8]:
n_epochs = 25
learning_rate = 1e-2

model = FeedForwardClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0.00)
scheduler = ExponentialLR(optimizer, gamma=0.9)

Training loop

In [9]:
for i in range(n_epochs):
     
    losses = []
    
    for batch, (X, y) in enumerate(train_loader):
                
        probs = model(X)
        
        loss = loss_fn(probs, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
    scheduler.step()

        
    if (i + 1) % 5 == 0:
        print(f"epoch {i + 1}, Train loss = {np.array(losses).mean():.3f} ")
        print()

epoch 5, Train loss = 1.856 

epoch 10, Train loss = 1.414 

epoch 15, Train loss = 1.048 

epoch 20, Train loss = 0.813 

epoch 25, Train loss = 0.681 



In [10]:
torch.save(model.state_dict(), "./model.pth")

Handle new user input

In [11]:
def proccess_new_input(prompt):
    prompt = preprocess(prompt)
    prompt = vectorizer.transform(prompt).toarray()
    prompt = torch.from_numpy(prompt).to(torch.float32)
    return(prompt)

Loop for interacting with the bot.

In [12]:
intents = intents['intents']


while (True):
    
    usr_input = [input()]
    
    processed_input = proccess_new_input(usr_input)
    
    probs = model(processed_input)
    
    predict = probs.argmax()
       
    responses = intents[predict]['responses']
    
    response = random.sample(responses, 1)[0]
    
    print(response)
    
    if (intents[predict]['tag'] == "goodbye"):
        break
    

 Hello


Hi. How can i help you?


 WHat do you have?


We have the newest novels.


 How can i pay?


We take paypal, venmo and cash!


 WHat would you recommend?


The Hitchhiker's Guide To The Galaxy by Douglas Adams


 Thanks bye


Have a nice day!
